<a href="https://colab.research.google.com/github/kamrulhasan04947/mlprac/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import pandas as pd
import numpy as np

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import os

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *

In [ ]:
wordnet = WordNetLemmatizer()

In [ ]:
def dataBuilder(path):
  nameOflabels = ['neg', 'pos']
  sentiment = []
  label = []
  if os.path.exists(path):
    for dirname in os.listdir(path):
      if dirname in nameOflabels:
        class_name = nameOflabels.index(dirname)
        datapath = os.path.join(path,dirname)
        for file in os.listdir(datapath):
          fl = open(os.path.join(datapath, file), 'r')
          sent_data = nltk.word_tokenize(re.sub('[^a-zA-Z]', ' ', fl.read()).lower())# removing punction and numbers extept words that have letters (a-z and A-Z ) # converting all words lower
          sent_data = [wordnet.lemmatize(word) for  word  in sent_data if word not in set(stopwords.words('english'))]
          sent_data = ' '.join(sent_data)
          sentiment.append(sent_data)
          label.append(class_name)
  return pd.DataFrame({'sentiments': sentiment, 'label': label})

In [ ]:
xtrain, ytrain= dataBuilder('/content/drive/MyDrive/dataset/data/train')
xtest , ytest= dataBuilder('/content/drive/MyDrive/dataset/data/test')

In [ ]:
# wordnet = WordNetLemmatizer()

# def pcontext_processor(context):
#   #tokenization applying
#   # find the sentences from paragraph
#   corpus = [] # creating an empty list
#   for i in range(0, len(context)): # iterating through sentences
#     sent_data = nltk.word_tokenize(re.sub('[^a-zA-Z]', ' ', context['sentiments'][i]).lower())# removing punction and numbers extept words that have letters (a-z and A-Z ) # converting all words lower
#     sent_data = [wordnet.lemmatize(word) for  word  in sent_data if word not in set(stopwords.words('english'))] # iterating through list of words to remove stop words and lamatized thest word by applying list comprehansion.
#     sent_data = ' '.join(sent_data) # gathering words in a itaretive form
#     corpus.append(sent_data) # appending sentence on corpus list
#   return corpus

In [ ]:
train_courpus = pcontext_processor(train_data)

In [ ]:
test_courpus = pcontext_processor(test_data)

In [ ]:
vocab_size = 10000  # Adjustable based on your vocabulary size
max_sequence_length = 20  # Adjustable based on your desired sequence length
embedding_dim = 100
lstm_units = 64
num_classes = 3  # Number of classes (positive, negative, neutral)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, datasets, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_courpus)
tokenizer.fit_on_texts(test_courpus)
train_sequence = tokenizer.texts_to_sequences(train_courpus)
test_sequence = tokenizer.texts_to_sequences(test_courpus)
padded_train_sequences = pad_sequences(train_sequence, maxlen=max_sequence_length, padding="post", truncating="post")
padded_test_sequences = pad_sequences(train_sequence, maxlen=max_sequence_length, padding="post", truncating="post")


In [ ]:
y_train = pd.get_dummies(train_data['label'])
y_test = pd.get_dummies(test_data['label'])

In [ ]:
y_train= y_train.iloc[:, 1].values
y_test = y_test.iloc[:, 1].values

In [ ]:
categorical_train_labels = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
categorical_test_labels = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

In [ ]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Conv1D(32, 5, activation='relu', input_shape=(64, )),
    layers.MaxPool1D(5),
    LSTM(lstm_units),
    tf.keras.layers.Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(padded_train_sequences, categorical_train_labels, epochs=10, batch_size=32)

Epoch 1/10
782/782 [==============================] - 20s 22ms/step - loss: 0.5814 - accuracy: 0.6866
Epoch 2/10
782/782 [==============================] - 17s 21ms/step - loss: 0.3817 - accuracy: 0.8358
Epoch 3/10
782/782 [==============================] - 17s 21ms/step - loss: 0.1641 - accuracy: 0.9400
Epoch 4/10
782/782 [==============================] - 16s 21ms/step - loss: 0.0404 - accuracy: 0.9873
Epoch 5/10
782/782 [==============================] - 16s 20ms/step - loss: 0.0180 - accuracy: 0.9940
Epoch 6/10
782/782 [==============================] - 16s 21ms/step - loss: 0.0147 - accuracy: 0.9953
Epoch 7/10
782/782 [==============================] - 16s 21ms/step - loss: 0.0113 - accuracy: 0.9960
Epoch 8/10
782/782 [==============================] - 17s 21ms/step - loss: 0.0099 - accuracy: 0.9969
Epoch 9/10
782/782 [==============================] - 17s 21ms/step - loss: 0.0087 - accuracy: 0.9975
Epoch 10/10
782/782 [==============================] - 17s 21ms/step - loss: 0.010

In [ ]:
score = model.evaluate(padded_test_sequences, categorical_test_labels, verbose = 3)

In [ ]:
print(score)

[14.269189834594727, 0.0012000000569969416]
